In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from preprocessing import *
from cnn_models import *
from datetime import datetime

%load_ext autoreload
%autoreload 2

#### Overview
The goal here is to use the CNN to reduce the size of the input image to obtain a "discretized" image of shape, e.g. (W/16, H/16). Every entry of this image is related to a patch in the input image. This obtained image is compared by the CNN with the groundtruth (after properly discretizing by it patch-wise).

### - Load data

In [ ]:
# Loaded a set of images
n = 2

imgs, gt_imgs = load_images(n)
imgs[0].shape, gt_imgs[0].shape

In [ ]:
# imgs_aug, gt_imgs_aug = rand_augment_images(imgs, gt_imgs)
# i = 2
# fig, axs = plt.subplots(1, 2)
# fig.set_size_inches((20, 10))
# axs[0].imshow(imgs_aug[i], cmap='gray')
# axs[1].imshow(gt_imgs_aug[i], cmap='gray')

### - Convert the data to the correct format
We reshape each input to fulfill the requirements of the tensorflow library. 

In [ ]:
# !!! set predict_patch_width in accordance to the model you are using !!!
# the shape of the output of the model depends on the strides parameters 
# (if a layer has stride=2 then each ouput's side is half of the input'side).
# predict_patch_width must be equal to the total reduction of the model, e.g.
# if the model has three layer with stride=2 => the input of the model is 
# reduced by a factor of 2*2*2=8, i.e. the ouptut will be patch-wise with 
# patches 8x8 pixels.
predict_patch_width = 8

X, Y = images_to_XY(imgs, gt_imgs, predict_patch_width=predict_patch_width)
X.shape, Y.shape

### - For now avoid cross validation, just split the datasest in test and train. 

In [ ]:
test_ratio = 0.5

train, test = split_train_test(X, Y, test_ratio=test_ratio, seed=1)
train.X.shape, train.Y.shape, test.X.shape, test.Y.shape 

In [ ]:
# # check it makes sense (show the i-th input of set_)
# i = 0
# set_ = test

# fig, axs = plt.subplots(1, 2, figsize=(20, 10))
# axs[0].imshow(set_.Y[i, :, :, 1], cmap='gray')
# axs[1].imshow(set_.X[i, :, :])

### - Build the CNN model or load a previous one

Choose one of the models you defined (model_n) and initialize it.

In [ ]:
# generate an unique name for the model (so to avoid overwriting previous models)
model_name = "model_"+str('{0:%Y-%m-%d_%H:%M:%S}'.format(datetime.now()))
model_path = "models/"+model_name
model = CnnModel(model_n = 0, model_path=model_path)
model.summary()

Otherwise load a previous model

In [ ]:
# give the folder
model_name = "model1_10_12"
model_path = "models/"+model_name
model = CnnModel(model_path=model_path)
model.load() # load the model and its weights
model.summary()

In [ ]:
model.compile()

### - Train the model on the train data while validating it on the test data

In [ ]:
num_epochs=5
batch_size=2
model.train(train, test, num_epochs, batch_size) 

#### Plot the accuracy and the loss obtained during training

In [ ]:
last_epochs=100 # plot only the last 100 epochs
model.plot_history(last_epochs=100)

#### Display the output of a specific layer

In [ ]:
# these are all the layers 
model.model.layers

In [ ]:
# choose a layer and an image 
image = test.X[0]
layer_num = 8

model.show_layer_output(image, layer_num, filename="") # pass a filename if you want to store the image to file 

### - Evaluate the model on the test data

In [ ]:
# check the performance on train or test
set_ = train

model.evaluate_model(set_.X, set_.Y)

### - Show a prediction

In [ ]:
# choose an image to predict (or part of it)
img = test.X[0][:, :]

model.display_prediction(img, ax=None)

### - Save/load model

In [ ]:
model.save()

### - Others

In [ ]:
# some callbacks example: 

# create a list of callbacks we want to use during training
# # a callback to store epoch results to a csv file
# filename='model_train_new.csv'
# csv_log = callbacks.CSVLogger(filename, separator=',', append=False)

# # a callback to stob before doing the predefined number of epochs (stop before overfitting the data)
# early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='min')

# # a callback to save the best model (best model = the one with the lowest 'monitor' variable)
# filepath = "best-weights-{epoch:03d}-{loss:.4f}-{acc:.4f}.hdf5"
# checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# # callbacks_list = [csv_log,early_stopping,checkpoint]

print(
    "-get configurations:", "\n",
    model.get_config(), "\n",
    model.layers[0].get_config(), "\n",

    "\n-get shapes", "\n",
    model.layers[0].input_shape, "\n",
    model.layers[0].output_shape, "\n",
    
    "\n-get weights", "\n",
    model.layers[0].get_weights()[0].shape, "\n",
    
    "\n-check if trainable", "\n",
    model.layers[0].trainable, "\n", # you can set this to false to "freeze" a layer
)

In [ ]:
from IPython.core.debugger import Pdb
debugger = Pdb()
debugger.set_trace() # put this line as a breakpoint